In [10]:
%pip install transformers torch datasets


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import json
import os
import numpy as np


def read_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print("File not found.")
        return None
    except json.JSONDecodeError:
        print("Invalid JSON format.")
        return None


FILE_PATH = ".//tokens//REMImini//0.json"
data = read_json_file(FILE_PATH)
if data:
    ids = data.get("ids")
    if ids:
        ids = ids[0]  # Remove one level of list
        ids_string = str(ids)  # Convert the list to a string
    ids_bpe_encoded = data.get("ids_bpe_encoded")
    print("IDs:", ids)
    # print("IDs as string:", ids_string)
    # print("IDs BPE Encoded:", ids_bpe_encoded)


IDs: [4, 189, 294, 206, 55, 107, 126, 209, 39, 103, 127, 210, 55, 106, 132, 212, 43, 106, 127, 215, 46, 105, 126, 218, 56, 111, 125, 51, 106, 127, 4, 189, 41, 108, 133, 58, 109, 126, 190, 56, 112, 125, 191, 58, 105, 125, 56, 107, 130, 192, 51, 107, 126, 194, 50, 107, 127, 197, 55, 109, 125, 198, 48, 101, 127, 56, 104, 125, 200, 58, 109, 136, 43, 105, 135, 203, 46, 108, 128, 206, 50, 108, 127, 209, 51, 107, 157, 212, 55, 109, 126, 215, 58, 112, 127, 218, 62, 112, 128, 43, 107, 127, 4, 189, 63, 111, 127, 192, 60, 111, 127, 44, 106, 126, 195, 63, 109, 127, 197, 58, 110, 127, 43, 107, 127, 200, 63, 108, 127, 203, 56, 110, 127, 204, 41, 104, 130, 206, 55, 110, 127, 209, 56, 111, 127, 50, 108, 129, 212, 65, 111, 127, 215, 56, 110, 127, 39, 109, 136, 218, 55, 110, 127, 4, 189, 53, 109, 127, 43, 108, 125, 192, 51, 108, 127, 195, 55, 110, 136, 46, 107, 128, 198, 38, 107, 127, 201, 51, 108, 125, 36, 105, 126, 203, 34, 108, 127, 207, 33, 107, 137, 48, 108, 128, 210, 60, 110, 127, 212, 53, 110, 12

In [13]:
TRAIN, VALID = 0.8, 0.1

train_tokens = ids[:int(len(ids) * TRAIN)]
valid_tokens = ids[int(len(ids) * TRAIN): int(len(ids)*(TRAIN+VALID))]
test_tokens = ids[int(len(ids) * (TRAIN+VALID)):]

print(len(train_tokens), len(valid_tokens), len(test_tokens))

23625 2953 2954


In [14]:
def read_files_in_directory(directory):
    data_dict = {}
    for i, filename in enumerate(os.listdir(directory)):
        with open(os.path.join(directory, filename), 'r') as file:
            data = json.load(file)
            ids = data.get("ids")
            if ids:
                data_dict[str(i)] = ids[0]  # Extract the first level of list from 'ids'
    return data_dict

DATA_DIR = ".//tokens//REMImini"
data_dict = read_files_in_directory(DATA_DIR)
print(data_dict)

{'0': [4, 189, 294, 206, 55, 107, 126, 209, 39, 103, 127, 210, 55, 106, 132, 212, 43, 106, 127, 215, 46, 105, 126, 218, 56, 111, 125, 51, 106, 127, 4, 189, 41, 108, 133, 58, 109, 126, 190, 56, 112, 125, 191, 58, 105, 125, 56, 107, 130, 192, 51, 107, 126, 194, 50, 107, 127, 197, 55, 109, 125, 198, 48, 101, 127, 56, 104, 125, 200, 58, 109, 136, 43, 105, 135, 203, 46, 108, 128, 206, 50, 108, 127, 209, 51, 107, 157, 212, 55, 109, 126, 215, 58, 112, 127, 218, 62, 112, 128, 43, 107, 127, 4, 189, 63, 111, 127, 192, 60, 111, 127, 44, 106, 126, 195, 63, 109, 127, 197, 58, 110, 127, 43, 107, 127, 200, 63, 108, 127, 203, 56, 110, 127, 204, 41, 104, 130, 206, 55, 110, 127, 209, 56, 111, 127, 50, 108, 129, 212, 65, 111, 127, 215, 56, 110, 127, 39, 109, 136, 218, 55, 110, 127, 4, 189, 53, 109, 127, 43, 108, 125, 192, 51, 108, 127, 195, 55, 110, 136, 46, 107, 128, 198, 38, 107, 127, 201, 51, 108, 125, 36, 105, 126, 203, 34, 108, 127, 207, 33, 107, 137, 48, 108, 128, 210, 60, 110, 127, 212, 53, 110, 1

In [15]:
import itertools
import csv
import numpy as np

# train_tokens = list(itertools.islice(data_dict.values(), int(len(data_dict) * TRAIN)))
# valid_tokens = list(itertools.islice(data_dict.values(), int(len(data_dict) * TRAIN), int(len(data_dict) * (TRAIN+VALID))))
# test_tokens = list(itertools.islice(data_dict.values(), int(len(data_dict) * (TRAIN+VALID)), int(len(data_dict))))
# all_tokens = list(data_dict.values())


train_tokens = list(itertools.islice(data_dict.items(), int(len(data_dict) * TRAIN)))
valid_tokens = list(itertools.islice(data_dict.items(), int(len(data_dict) * TRAIN), int(len(data_dict) * (TRAIN+VALID))))
test_tokens = list(itertools.islice(data_dict.items(), int(len(data_dict) * (TRAIN+VALID)), int(len(data_dict))))
all_tokens = list(data_dict.items())


# print(len(train_tokens), len(valid_tokens), len(test_tokens))
# print("Train")
# for value in train_tokens:
#     print(value)
# print(type(train_tokens))
# print("Valid")
# for value in valid_tokens:
#     print(value)
# print("Test")
# for value in test_tokens:
#     print(value)

In [16]:
def save_as_csv(data_list, filename):
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        for value in data_list:
            writer.writerow([value])

# save_as_csv(train_tokens, ".//data//train.csv")
# save_as_csv(valid_tokens, ".//data//valid.csv")
# save_as_csv(test_tokens, ".//data//test.csv")

# save_as_csv(all_tokens, ".//data//all.csv") 

In [17]:
from datasets import load_dataset

def save_as_jsonl(data_dict, filename):
    with open(filename, 'w') as f:
        for key, value in data_dict:
            json_data = {str(key): value}
            f.write(json.dumps(json_data) + "\n")

save_as_jsonl(train_tokens, ".//data//train.jsonl")
save_as_jsonl(valid_tokens, ".//data//valid.jsonl")
save_as_jsonl(test_tokens, ".//data//test.jsonl")
save_as_jsonl(all_tokens, ".//data//all.jsonl")

print(type(train_tokens))

<class 'list'>


In [18]:
# data_files = {"train": ".//data//train.jsonl",
#               "validation": ".//data//valid.jsonl", 
#               "test":".//data//test.jsonl"}

# dataset = load_dataset("csv", data_files=data_files)

In [19]:
import pandas as pd
import datasets

df = pd.DataFrame(data=all_tokens)
df.columns = ['file', 'tokens']

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)



# dataset.train_test_split(test_size=0.2)

   file                                             tokens
0     0  [4, 189, 294, 206, 55, 107, 126, 209, 39, 103,...
1     1  [4, 189, 294, 206, 27, 103, 125, 15, 101, 125,...
2     2  [4, 189, 294, 190, 17, 97, 125, 24, 103, 125, ...
3     3  [4, 189, 294, 206, 56, 107, 126, 212, 54, 105,...
4     4  [4, 189, 294, 205, 32, 113, 179, 20, 107, 179,...
5     5  [4, 189, 294, 206, 62, 104, 160, 50, 102, 163,...
6     6  [4, 189, 294, 205, 49, 101, 125, 206, 51, 101,...
7     7  [4, 189, 294, 205, 23, 108, 125, 24, 108, 125,...
8     8  [4, 189, 294, 205, 41, 109, 147, 34, 106, 145,...
9     9  [4, 189, 294, 202, 63, 114, 188, 51, 114, 188,...
10   10  [4, 189, 294, 190, 56, 108, 125, 198, 51, 109,...
11   11  [4, 189, 294, 205, 25, 104, 151, 209, 29, 104,...
12   12  [4, 189, 294, 205, 22, 102, 129, 209, 29, 103,...
13   13  [4, 189, 294, 206, 39, 117, 125, 27, 117, 125,...
14   14  [4, 189, 294, 205, 16, 97, 128, 208, 59, 104, ...
15   15  [4, 189, 294, 205, 48, 103, 126, 220, 48, 104,.

In [28]:
df_train = pd.DataFrame(data=train_tokens)
df_train.columns = ['file', 'tokens']

df_test = pd.DataFrame(data=test_tokens)
df_test.columns = ['file', 'tokens']

df_valid = pd.DataFrame(data=valid_tokens)
df_valid.columns = ['file', 'tokens']

train_dataset = datasets.Dataset.from_pandas(df_train)
test_dataset = datasets.Dataset.from_pandas(df_test)
valid_dataset = datasets.Dataset.from_pandas(df_valid)


REMImini_dict = datasets.DatasetDict({"train":train_dataset,"test":test_dataset, "valid":valid_dataset})

type(REMImini_dict['train'][0]['tokens'])

list

In [21]:
%pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [22]:
#jak na wmm - do skutku restart środowiska
from huggingface_hub import notebook_login

notebook_login()

In [29]:
REMImini_dict.push_to_hub('REMImini_set')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/TayTT/REMImini_set/commit/0fc39a4f53b1875c18e64da55e60719976c092b1', commit_message='Upload dataset', commit_description='', oid='0fc39a4f53b1875c18e64da55e60719976c092b1', pr_url=None, pr_revision=None, pr_num=None)